In [1]:
import pandas as pd
import MeCab
import sys
from collections import Counter
import re
import urllib.request, urllib.error

In [2]:
tagger = MeCab.Tagger("-Owakati")
tagger.parse("") #バグ防止　空白をいれる

'\n'

In [38]:
df = pd.read_csv("../pretest_imamura_lunch.csv")

In [39]:
review = df["lunch_review"][0]
review

'同業者も足繁く通う白金高輪の名店でゆったりと寛いだ時間を過ごすことが出来、私の馬鹿舌も感覚が敏感に変化したかもしれない。'

In [32]:
df["dinner_review"]

0      白金の【鮨 いまむら】さんに伺いました。場所は地下鉄白金高輪駅を出て歩いて10分ちょっと。北...
1      今日は後輩くんの卒業式でいいお寿司を！仕込みされてるお店をいくつか厳選したどり着いたお店。白...
2      本日訪問したのは鮨いまむら。この日のためにGWを早めに切り上げて東京へ帰ってきました。ミシュ...
3      白金高輪にあるお鮨屋さん。駅からは一本道なので歩いて１０分くらいでわかりやすい。健康のために...
4      食べログ★4.16・2019年 bronze・2018年 bronze・2017年 bron...
5      久々のいまむらさん。この日は、同じビルで働く某グルメブロガーさんと。途中、まさかの仕事の呼び...
6      2018/4/18（水）dinnerinstagram「＠mitomi_emon」／ブログ「...
7      2年前くらいからずっと気になっていたお鮨屋さん。白金の住宅街に佇むお店です。最寄りは白金高輪...
8      白金高輪　鮨いまむら初訪問つまみからスタート日本酒大吟醸　富山のはねやひらめ蛸毛ガニ　ジュレ...
9      近頃評判の高いこちらの店に訪れてみました。こちらの店はとにかくネタが厚く大ぶりに切られ、文字...
10     友達が突然、「鮨が食べたい！」と、いうわけで1ヶ月ほど前にこちらを予約。丁度1ヶ月前に電話し...
11     2017.12白金という少し自宅から遠い場所にある為、気になっていたものの未訪のまま。何とか...
12     今回は平日の夜18時からの予約。仕事を終え、急いでタクシーで白金へと向かうと、開店前、若い女...
13     家の近所にありながら行きたくても行けなかったいまむら。日夜20時15分からの回でようやく予約...
14     東京 白金にある鮨 いまむらさんに夕食を頂きに行きました。白金高輪駅より徒歩10分程度です。...
15     みんな！シースーマンだよ！もう2ヶ月半くらい前になるけど、ずっと行きたかった鮨いまむら@白金...
16     東京メトロの白金高輪駅の３番出口を出て、駅を背にして右方面に歩き、突き当たりを左折してすぐの...
17     まず、こちらはサービス料無しで夜は2部制昼は土日

In [6]:
stop_words = [ "○－", "○○/", ")○/",
                 "～」", "○－", "○○/", ")○/", "～。", "○/", "://", "com", '）"', ",\u3000", "\u3000-",
                "◎/", "◎◎/", "）⚪", ")、", "^)", "(^", "cp", "ーー",
                '""', "！⚫",]

In [27]:
def normalize_number(text):
    # 連続した数字を0で置換
    #replaced_text = re.sub(r'\d+', '0', text)
    return text

In [42]:
def tokenize_ja(text, lower):
    #text = replaced_text = re.sub(r"\d+", "0", text) 
    # 連続した数字を0で置換
    node = tagger.parseToNode(str(text)) #分かち書きしたのが入っている。イテレータ
    while node:
        surface = node.surface.lower() #surfaceは単語の見た目　例：トろ
        if lower and node.feature.split(',')[0] in ["名詞","形容詞", "動詞", "副詞", "助詞", "助動詞", "接続詞", "連体詞", "感動詞", "接頭詞"] and surface not in stop_words:
            """
            feature = "動詞,食べる"
            feature.split(",")
            >>>['動詞', '食べる']
            feature.split(",")feature.split(",")[0]
            >>>'動詞'
            """
            
            #分かち書きで取得する品詞を指定
            yield surface 
            #単語の見た目
            #lowerで小文字に変換 正規化
        node = node.next

In [29]:
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len
    ]

In [44]:
review = df["lunch_review"][1]
review

'ランチ@鮨いまむら2018年10月15日(月)17時13分17秒NEW!テーマ：レストラン昨日のランチは大好きなお寿司！白金高輪駅より、北里通りへ進み北里大学病院すぐ手前にある鮨いまむらへ。美人女将が暖簾を出すタイミングに伺いました。\xa0\xa0\xa0築地が移転してからのお話も少し伺いましたよ。場外のお店は移転しておらず、場外での買い物もあるので、豊洲と築地に行かねばならないのが少し道が混んで大変とか。豊洲自体は広く、パーキングはいっぱいだけど、停める場所がたくさんあるそう。ほー、初日だけ大変だったようで後は順調の様子です。\xa0\xa0グラスで白ワインを。シャブリです。昨夜のボトルかな？口開けではなかったけど、風味はまずまずでした。\xa0■スターターはカマス。皮目をバリッと焼き上げて。一味、大根おろし、ポン酢で。\xa0\xa0■茶碗蒸し松茸入りなのですが…黒胡椒が…非常に効いて。松茸の香りが吹っ飛ぶほど（笑）。辛かった。無くても良いかな。\xa0\xa0■白子焼きどうしても、ししゃも焼きを思い出す香りがして（笑）濃厚で美味しいのですが、脳裏にししゃも。\xa0■握りお任せでお願いしました。お腹いっぱいで苦しいのに、トロたくは外せず。カステラのような卵焼き、甘いアナゴ、ウニ。\xa0\xa0アジの分厚いこと！女性にはちと苦しいほど。車海老に貝も歯応えがありgood.\xa0\xa0高すぎない軍艦が嬉しいイクラ、まるで柄のようなコハダに厚いイカ、酸味のバランス良いトロ。\xa0\xa0■デザートは小ぶりなきな粉アイス。黒みつとの相性も抜群でした。\xa0ご馳走さま。'

In [45]:
tokenize(review, 1, 10000, True)

['ランチ',
 '@',
 '鮨',
 'いま',
 'むら',
 '2018',
 '年',
 '10',
 '月',
 '15',
 '日',
 '(',
 '月',
 ')',
 '17',
 '時',
 '13',
 '分',
 '17',
 '秒',
 'new',
 '!',
 'テーマ',
 'レストラン',
 '昨日',
 'の',
 'ランチ',
 'は',
 '大好き',
 'な',
 'お',
 '寿司',
 '白金',
 '高輪',
 '駅',
 'より',
 '北里',
 '通り',
 'へ',
 '進み',
 '北里大学',
 '病院',
 'すぐ',
 '手前',
 'に',
 'ある',
 '鮨',
 'いま',
 'むら',
 'へ',
 '美人',
 '女将',
 'が',
 '暖簾',
 'を',
 '出す',
 'タイミング',
 'に',
 '伺い',
 'まし',
 'た',
 '築地',
 'が',
 '移転',
 'し',
 'て',
 'から',
 'の',
 'お話',
 'も',
 '少し',
 '伺い',
 'まし',
 'た',
 'よ',
 '場外',
 'の',
 'お',
 '店',
 'は',
 '移転',
 'し',
 'て',
 'おら',
 'ず',
 '場外',
 'で',
 'の',
 '買い物',
 'も',
 'ある',
 'ので',
 '豊洲',
 'と',
 '築地',
 'に',
 '行か',
 'ね',
 'ば',
 'なら',
 'ない',
 'の',
 'が',
 '少し',
 '道',
 'が',
 '混ん',
 'で',
 '大変',
 'と',
 'か',
 '豊洲',
 '自体',
 'は',
 '広く',
 'パーキング',
 'は',
 'いっぱい',
 'だ',
 'けど',
 '停める',
 '場所',
 'が',
 'たくさん',
 'ある',
 'そう',
 'ほ',
 'ー',
 '初日',
 'だけ',
 '大変',
 'だっ',
 'た',
 'よう',
 'で',
 '後',
 'は',
 '順調',
 'の',
 '様子',
 'です',
 'グラス',
 'で',
 '白',
 'ワイン',
 'を',
 'シャブリ',
 'です',
 '昨

In [ ]:
for i in df["lun"]